In [2]:
import sqlite3
import pandas as pd
import numpy as np
import sys

In [3]:
RT_LIMIT_LOWER = 4340
RT_LIMIT_UPPER = 4580

In [4]:
BASE_NAME = "/home/ubuntu/HeLa_20KInt-rt-{}-{}".format(RT_LIMIT_LOWER,RT_LIMIT_UPPER)

In [5]:
import pickle
allpeptides_subset_df = pd.read_pickle("{}/allpeptides_subset_df.pkl".format(BASE_NAME))

In [6]:
allpeptides_subset_df.head()

,mz,intensity,charge_state,rt,scan,mq_feature_id
273238,895.95670,26707000.0,2,4432.552,363,1
271311,877.90152,16367000.0,2,4457.345,411,2
258239,830.45670,10780000.0,2,4398.299,435,3
272371,890.44758,9524100.0,2,4512.851,348,4
257004,821.89394,8506200.0,2,4536.471,465,5


In [7]:
evidence_df = pd.read_csv('/home/ubuntu/maxquant_results/txt/evidence.txt', sep='\t')

In [8]:
evidence_df.head()

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Missed cleavages,Proteins,...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,AIF MS/MS IDs,Oxidation (M) site IDs
0,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42|PABP2_HUMAN,...,NaN,NaN,0,2971,0,0,0,0,NaN,NaN
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,0,0,0,sp|P37108|SRP14_HUMAN;tr|H0YLW0|H0YLW0_HUMAN,...,NaN,NaN,1,1396,1,1,1,1,NaN,NaN
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,0,0,0,sp|P37108|SRP14_HUMAN;tr|H0YLW0|H0YLW0_HUMAN,...,NaN,NaN,2,1396,1,1,2,2,NaN,NaN
3,AAAAAAAAVPSAGPAGPAPTSAAGR,25,Unmodified,_AAAAAAAAVPSAGPAGPAPTSAAGR_,NaN,NaN,0,0,0,sp|Q9Y4H2|IRS2_HUMAN,...,NaN,NaN,3,4916,2,2,3,3,NaN,NaN
4,AAAAAAAGDSDSWDADAFSVEDPVRK,26,Acetyl (Protein N-term),_(ac)AAAAAAAGDSDSWDADAFSVEDPVRK_,NaN,NaN,1,0,1,sp|O75822|EIF3J_HUMAN,...,NaN,NaN,4,595,3,3,4,4,NaN,NaN


In [9]:
evidence_df.columns

Index(['Sequence', 'Length', 'Modifications', 'Modified sequence',
       'Oxidation (M) Probabilities', 'Oxidation (M) Score Diffs',
       'Acetyl (Protein N-term)', 'Oxidation (M)', 'Missed cleavages',
       'Proteins', 'Leading proteins', 'Leading razor protein', 'Type',
       'Raw file', 'MS/MS m/z', 'Charge', 'm/z', 'Mass', 'Resolution',
       'Uncalibrated - Calibrated m/z [ppm]',
       'Uncalibrated - Calibrated m/z [Da]', 'Mass error [ppm]',
       'Mass error [Da]', 'Uncalibrated mass error [ppm]',
       'Uncalibrated mass error [Da]', 'Max intensity m/z 0', 'Retention time',
       'Retention length', 'Calibrated retention time',
       'Calibrated retention time start', 'Calibrated retention time finish',
       'Retention time calibration', 'Ion mobility index',
       'Ion mobility length', 'CCS', 'CCS length', 'Match time difference',
       'Match m/z difference', 'Match q-value', 'Match score',
       'Number of data points', 'Number of scans', 'Number of isotopic

In [10]:
allpeptides_df = pd.read_csv('/home/ubuntu/maxquant_results/txt/allPeptides.txt', sep='\t')

In [11]:
allpeptides_df.columns

Index(['Raw file', 'Charge', 'm/z', 'Mass', 'Resolution',
       'Number of data points', 'Number of frames', 'Number of isotopic peaks',
       'Isotope correlation', 'Mass fractional part', 'Mass deficit',
       'Retention time', 'Retention length', 'Retention length (FWHM)',
       'Min frame index', 'Max frame index', 'Ion mobility index',
       'Ion mobility index length', 'Ion mobility index length (FWHM)',
       'Intensity', 'Intensities', 'Number of pasef MS/MS', 'Pasef MS/MS IDs',
       'MS/MS scan number'],
      dtype='object')

In [58]:
MQ_FEATURE_ID = 42

In [59]:
mq_feature_df = allpeptides_subset_df[allpeptides_subset_df.mq_feature_id==MQ_FEATURE_ID]
mq_feature_df

,mz,intensity,charge_state,rt,scan,mq_feature_id
139523,621.97655,2352200.0,3,4438.456,714,42


In [60]:
mz = mq_feature_df.mz.values[0]
intensity = mq_feature_df.intensity.values[0]
charge_state = mq_feature_df.charge_state.values[0]
rt = mq_feature_df.rt.values[0]
scan = mq_feature_df.scan.values[0]

In [61]:
match_df = evidence_df[
    (evidence_df['Intensity'] == intensity) & (evidence_df['Ion mobility index'] == scan)
]
match_df

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Missed cleavages,Proteins,...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,AIF MS/MS IDs,Oxidation (M) site IDs
15092,GQYISPFHDIPIYADK,16,Unmodified,_GQYISPFHDIPIYADK_,NaN,NaN,0,0,0,sp|Q15181|IPYR_HUMAN;tr|Q5SQT6|Q5SQT6_HUMAN,...,NaN,NaN,15092,2440,11789,11878,15092,15092,NaN,NaN
